# Regression model to predict the house price

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error, r2_score

In [ ]:
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = 12, 8

### Reading data

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train_data = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test_data = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

Train set is comparable in size to the test set. Target column ('SalePrice') is present only in the train set.

In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
train_data.head()

### Defining target value and input features

This competition requires to predict total price of the house based on the given numeric and categorical features. In appraisal industry it is generally considered the best approach to estimate the price of real estate per sq.m or sq.ft. Total price is then calculated by multiplying the price per sq.m or sq.ft by the floor area. The reason for that being that size of the house in itself is a parameter, which affects the price per sq.m (the larger the house - the smaller the price per sq.m).

Price per sq.m also depends on the number of floors in the house. As a rule, the larger area is being concentrated on the first floor - the higher the price. Multi-storey houses with large basements are priced lower per sq.m than one-storey houses of similar total floor area.

In [ ]:
# Total square footage of the house (basement + 1st floor + 2nd floor)
train_data['TotalSF'] = train_data['TotalBsmtSF'] + train_data['1stFlrSF'] + train_data['2ndFlrSF']

In [ ]:
# Transform the original data for area in sq.ft on each floor into respective share of the total floor area
for column in ('TotalBsmtSF', '1stFlrSF', '2ndFlrSF'):
    train_data[column] = train_data[column] / train_data['TotalSF']

In [ ]:
# Transform measument of unfinished area in the basement into its share of the total basement area
train_data['BsmtUnfSF'] = train_data['BsmtUnfSF'] / train_data['TotalBsmtSF']

In [ ]:
# Transform measument of low quality area on all floors into its share of the total floor area
train_data['LowQualFinSF'] = train_data['LowQualFinSF'] / train_data['TotalSF']

In [ ]:
# Target value will be modified to price per sq.ft of total area
y_train = train_data['SalePrice'] / train_data['TotalSF']

In [ ]:
# Input features with the exception of the sale price and object ID
x_train = train_data.drop(['Id', 'SalePrice'], axis='columns')

### Dealing with missing values and categorical features

The data requires preprocessing in tearms of dealing with missing values and categorical features. Shuffling the train samples in this case is not necessary because original data is not sorted in any way.

All input features that are available in the dataset are relevant to the purpose of pricing the house, though they vary in their significance. The best approach would be to use as much original features as possible and fit the data to some kind of decision tree algorithm, which automatically detects the most important features.

In [ ]:
# Checking how many values are missing
missing_vals = train_data.isna().sum()
print(missing_vals[missing_vals > 0])

Some of the input features have high share of missing values, which makes them less useful for the model. These features are relatively few, and judging by their names we can safely assume they are not crucial. Only the features with less than 100 missing values will be retained for the further analysis.

In [ ]:
# Drop columns with more than 100 missing values
columns_to_drop = missing_vals[missing_vals > 100].index
x_train = x_train.drop(columns_to_drop, axis='columns')
print(columns_to_drop)

Missing values in the remaining columns will be imputed using 'median' strategy for numeric columns and 'most_frequent' strategy for categorical columns. Some of the columns that do not contain NaNs in the train set have missing values in the test set. To simplify and automate data processing we add numeric and categorical features to lists, which will be used in a column transformer as part of the data processing and modelling pipeline.

In [ ]:
# Columns by data type
cat_columns = list(x_train.select_dtypes(include=['category', 'object']).columns)
num_columns = list(x_train.select_dtypes(include=['number']).columns)

In [ ]:
# Imputing missing values, one-hot-encoding categorical columns.
# Scaler is not being used because RandomForest algorithm does not require scaling numeric values.
imp_median = SimpleImputer(strategy='median')
imp_freq = SimpleImputer(strategy='most_frequent')
ohe = OneHotEncoder(sparse=False)

cat_transformer = make_pipeline(imp_freq, ohe)

col_transformer = make_column_transformer((imp_median, num_columns),
                                          (cat_transformer, cat_columns),
                                          remainder='passthrough')

### Making a pipeline for regression model

In [ ]:
# Combine column transformer with the regression model
regr_model = RandomForestRegressor()
pipe_RF = make_pipeline(col_transformer, regr_model)

In [ ]:
# Training the model
pipe_RF.fit(x_train, y_train)

### Evaluating the model on the train set

In practice when appraisal specialists create evaluation models they tend to filter out outliers - objects with uptypically high or low prices, that could not be justified by the stardard regression model. Outliers in the real estate market usually belong to one of the following groups:
- unique properties with rare features that are not captured in the stardard description;
- properties sold on special conditions (seller looking for a quick deal, etc.);
- errors in property description.

Taking that into account, the optimal metrics to evaluate the house price model would be R2 (determination coefficient) and MAE (Mean Absolute Error). MSE (Mean Squared Error) in this case should not be considered as it amplifies the importance of outliers.

In [ ]:
# Predicting price per sq.ft
y_pred = pipe_RF.predict(x_train)

In [ ]:
# Evaluating the errors based on actual and predicted price per sq.ft
MAE = mean_absolute_error(y_pred, y_train)
R2 = r2_score(y_pred, y_train)
print(f'RandomForest price prediction per sq.ft\nMAE = {MAE}\nR2 = {R2}')

In [ ]:
# Average value of the predicted parameter and MAE to average value ratio
av_price_sqft = np.mean(y_train)
print(f'Average price per sq.ft = {av_price_sqft}\nMAE/av.price = {MAE/av_price_sqft}')

In [ ]:
# Calculating total price of the house
train_result = pd.DataFrame({'PredPriceSF': y_pred, 'TotalSF': train_data['TotalSF']})
train_result['PredPrice'] = train_result['PredPriceSF'] * train_result['TotalSF']

In [ ]:
train_result.head()

In [ ]:
# Evaluating the errors based on actual and predicted price for the house
MAE = mean_absolute_error(train_result['PredPrice'], train_data['SalePrice'])
R2 = r2_score(train_result['PredPrice'], train_data['SalePrice'])
print(f'RandomForest price prediction\nMAE={MAE}\nR2={R2}')

In [ ]:
# Average price of the house in the train set and MAE to average value ratio
av_house_price = train_data['SalePrice'].mean()
print(f'Average house price = {av_house_price}\nMAE/av.price = {MAE/av_house_price}')

In [ ]:
# Absolute errors
train_result['SalePrice'] = train_data['SalePrice']
train_result['AbsError'] = abs(train_result['PredPrice'] - train_result['SalePrice'])

In [ ]:
# Distribution of errors and their magnitude
plt.hist(train_result['AbsError'], bins=20, log=True)
plt.axvline(av_house_price, color='red', label='Average Price', linewidth=2)
plt.legend()
plt.xlabel('Absolute error')
plt.ylabel('Frequency, log scale')
plt.title('Distribution of Prediction Errors')
plt.show()

The histogram shows that large-scale errors are rare for the model and most likely caused by some errors in the original data or non-typical objects present in the dataset. These objects represent minor share of the training set, however they can significantly distort and skew the regression model.

To improve the model quality and robustness to extreme outliers we can eliminate them from the training set altogether. In this case the model will predict prices for typical houses more accurately but errors in regards to extreme outliers will not be eliminated. For practical business purposes this approach would be more useful than overfitting the model on nontypical data samples.

In [ ]:
outliers = train_result[train_result['AbsError'] > 200_000]
outliers_indexes = outliers.index
outliers

In [ ]:
# Removing outliers from input data and targets
x_train.drop(outliers_indexes, inplace=True)
y_train.drop(outliers_indexes, inplace=True)

In [ ]:
# Retraining the model on data cleaned from extreme outliers
pipe_RF.fit(x_train, y_train)

In [ ]:
# Predicting price per suare foot
y_pred = pipe_RF.predict(x_train)

In [ ]:
# Evaluating the errors based on actual and predicted price per sq.ft
MAE = mean_absolute_error(y_pred, y_train)
R2 = r2_score(y_pred, y_train)
print(f'RandomForest price prediction per sq.ft\nMAE = {MAE}\nR2 = {R2}')

To improve prediction accuracy we can train other regression models on the same data cleaned from extreme outliers with the same preprocessing steps and compare the metrics with results demonstrated by the RandomForest model.

We will check the accuracy of GradientBoosting model. Unlike RandomForest, which can work with unscaled numeric data, for this model we will modify the column transformer to include scaling for numeric features.

In [ ]:
# Modifying the column transformer
scaler = StandardScaler()
num_transformer = make_pipeline(imp_median, scaler)
col_transformer_scaled = make_column_transformer((num_transformer, num_columns),
                                                 (cat_transformer, cat_columns),
                                                 remainder='passthrough')

In [ ]:
# Training GradientBoosting model
pipe_GB = make_pipeline(col_transformer_scaled, GradientBoostingRegressor())
pipe_GB.fit(x_train, y_train)

In [ ]:
# Evaluating the model
pipe_GB.predict(x_train)
MAE = mean_absolute_error(y_pred, y_train)
R2 = r2_score(y_pred, y_train)
print(f'GradientBoosting prediction per sq.ft.\nMAE = {MAE}\nR2 = {R2}')

GradientBoosting model did not improved the prediction accuracy. For this task both models are appropriate.

### Making prediction on the test set

In [ ]:
# One of the test data samples contains NaN value in the 'TotalBsmtSF' column
test_data['TotalBsmtSF'].isna().sum()

In [ ]:
# We can assume in this case that missing value means there is no basement in the house
test_data['TotalBsmtSF'] = test_data['TotalBsmtSF'].fillna(0)

In [ ]:
# Calculate total floor area of the houses in the test set
# to be able to get the sale price from predicted price per sq.ft
test_data['TotalSF'] = test_data['TotalBsmtSF'] + test_data['1stFlrSF'] + test_data['2ndFlrSF']

In [ ]:
# Transform the original data for area in sq.ft on each floor into respective share of the total floor area
for column in ('TotalBsmtSF', '1stFlrSF', '2ndFlrSF'):
    test_data[column] = test_data[column] / test_data['TotalSF']

In [ ]:
# Transform measument of unfinished area in the basement into its share of the total basement area
test_data['BsmtUnfSF'] = test_data['BsmtUnfSF'] / test_data['TotalBsmtSF']

In [ ]:
# Transform measument of low quality area on all floors into its share of the total floor area
test_data['LowQualFinSF'] = test_data['LowQualFinSF'] / test_data['TotalSF']

In [ ]:
# Drop unnecessary columns from the test data
x_test = test_data.drop('Id', axis='columns')
x_test = x_test.drop(columns_to_drop, axis='columns')

In [ ]:
# Predicted price per sq.ft.
y_pred = pipe_RF.predict(x_test)

In [ ]:
# Calculate total price based on the predicted price per sq.ft.
test_data['PredPriceSF'] = y_pred
test_data['SalePrice'] = test_data['TotalSF'] * test_data['PredPriceSF']

In [ ]:
# Select data for submission
submission = test_data[['Id', 'SalePrice']]
submission.head()

In [ ]:
# Saving the results to csv file
submission.to_csv('submission.csv', index=False)